## Run The Cell Below

In [1]:
#PyTorchの初期化
import torch
import torchvision

### GPIO imports
# GPIO is mainly used for controlling lights
# and recieving signals from a button
#
# > Note: GPIO needs to be set every 
#   time when the code bloack is run.
###
import Jetson.GPIO as GPIO

# GPIO initialize
GPIO.setwarnings(False)
GPIO.setmode( GPIO.BCM )

# pins initialize
CHECK_PIN = 5
BG_PIN = 6
ROCK_PIN = 13
SCIS_PIN = 19
PAPER_PIN = 26
VCC0_PIN = 12
VCC1_PIN = 16
VCC2_PIN = 20
VCC3_PIN = 21
LOW = GPIO.LOW#False
HIGH = GPIO.HIGH#True

GPIO.setup( [CHECK_PIN, BG_PIN, ROCK_PIN, SCIS_PIN, PAPER_PIN, VCC0_PIN, VCC1_PIN, VCC2_PIN, VCC3_PIN], GPIO.OUT )
GPIO.output( VCC0_PIN, HIGH )
GPIO.output( VCC1_PIN, HIGH )
GPIO.output( VCC2_PIN, HIGH )
GPIO.output( VCC3_PIN, HIGH )

GPIO.output( CHECK_PIN, HIGH )
GPIO.output( BG_PIN, HIGH )
GPIO.output( ROCK_PIN, HIGH )
GPIO.output( SCIS_PIN, HIGH )
GPIO.output( PAPER_PIN, HIGH )

#ランダム関数
import random

model = torchvision.models.alexnet(pretrained=False)

#種類を増やした場合、変更するところ
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

#モデルの読み込み
model.load_state_dict(torch.load('best_model.pth'))

#カメラのフォーマットを学習済みモデルのフォーマットに合わせ変換
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#カメラ画像の取得
from jetcam.usb_camera import USBCamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=224, height=224)
image = camera.read()
image_widget = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(image)

#カメラ画像の更新
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

#GPU
device = torch.device('cuda')
model = model.to(device)


#推論
import torch.nn.functional as F
import time
import sys
a=0
one_blocked=0.0
two_blocked=0.0
three_blocked=0.0
bg_blocke=0.0
def update(change):
    global blocked_slider, robot,a
    global one_blocked, two_blocked, three_blocked,bg_blocked
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    one_blocked = float(y.flatten()[0])
    two_blocked = float(y.flatten()[1])
    three_blocked = float(y.flatten()[2])
    bg_blocked = float(y.flatten()[3])
    values = [one_blocked, two_blocked, three_blocked, bg_blocked]
    a = values.index(max(values))
    #0　グー rock 
    #1　パー paper
    #2　チョキ scissors
    #3 background
    
    time.sleep(0.001)
        
update({'new': image})  # we call the function once to intialize


### import randoms
import math

### import colors
from colorama import Fore, Back, Style

### Functions

def name_of_value(val):
    if val == 1:
        GPIO.output( BG_PIN, HIGH )
        GPIO.output( ROCK_PIN, LOW )
        GPIO.output( SCIS_PIN, HIGH )
        GPIO.output( PAPER_PIN, HIGH )
        return "Rock    ";
    if val == 2:
        GPIO.output( BG_PIN, HIGH )
        GPIO.output( ROCK_PIN, HIGH )
        GPIO.output( SCIS_PIN, HIGH )
        GPIO.output( PAPER_PIN, LOW )
        return "Paper   ";
    if val == 3:
        GPIO.output( BG_PIN, HIGH )
        GPIO.output( ROCK_PIN, HIGH )
        GPIO.output( SCIS_PIN, LOW )
        GPIO.output( PAPER_PIN, HIGH )
        return "Scissor ";
    if val == 0:
        GPIO.output( BG_PIN, LOW )
        GPIO.output( ROCK_PIN, HIGH )
        GPIO.output( SCIS_PIN, HIGH )
        GPIO.output( PAPER_PIN, HIGH )
        return "Background ";
    
### MAIN

# activate hand gesture recognition
display(image_widget)
camera.observe(update, names='value')

# main process

GPIO.output( CHECK_PIN, LOW )
TIME_DELTA = 0.5
try:
    while True: # forever loop
        # Wait a little and detect hand gesture
        time.sleep( TIME_DELTA )
        sint = a;
        sys.stdout.write("\rYou: %s \tR:%f, P:%f, S:%f, BG:%f" % (name_of_value(sint), two_blocked, three_blocked, bg_blocked, one_blocked))
        sys.stdout.flush()
        
except KeyboardInterrupt:
    GPIO.output( CHECK_PIN, HIGH )
    GPIO.output( BG_PIN, HIGH )
    GPIO.output( ROCK_PIN, HIGH )
    GPIO.output( SCIS_PIN, HIGH )
    GPIO.output( PAPER_PIN, HIGH )
    GPIO.cleanup()
    sys.stdout.write(Style.RESET_ALL + "\n")
    sys.exit(0)
   

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

You: Scissor  	R:0.153009, P:0.187181, S:0.499478, BG:0.160331943

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "<ipython-input-1-2ef65f71853e>", line 181, in <module>
    time.sleep( TIME_DELTA )
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2ef65f71853e>", line 194, in <module>
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 319, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 353, in _fixed

SystemExit: 0

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
